In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
#Service_port con los planes
Service_port = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/MA5608T_Service_Port.xlsx')
#ONU con los Alias y contratos
onus = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/MA5608T_GPON_ONU.xlsx')



La informacion manejada por el software de gestion de la OLT contiene dos partes:
1. Service port: Maneja los planes de internet de los usuarios pero no contiene informacion del numero de contrato
2. GPON ONUs: Contiene la informacion de contrato de los usuarios pero no contiene la informacion de los planes de internet

Como ambas partes tienen la informacion del puerto de la olt a la que estan conectados los usuarios, la idea es comparar la informacion comun y crear un dataset que contenga el numero de contrato y su respectiva velocidad de internet

In [ ]:
print(Service_port)

                                      Name       Alias Connection Type  \
0      33/0_1_3/17/1/Multi-Service VLAN/33       73303        LAN-GPON   
1      33/0_1_0/45/1/Multi-Service VLAN/33        8175        LAN-GPON   
2      33/0_1_0/41/1/Multi-Service VLAN/33       10561        LAN-GPON   
3      33/0_1_1/74/1/Multi-Service VLAN/33        8192        LAN-GPON   
4      33/0_1_1/14/1/Multi-Service VLAN/33        8181        LAN-GPON   
..                                     ...         ...             ...   
950   31/0_1_11/64/1/Multi-Service VLAN/31    4497 -P4        LAN-GPON   
951    31/0_1_9/28/1/Multi-Service VLAN/31  600008 -P7        LAN-GPON   
952    31/0_1_4/27/1/Multi-Service VLAN/31    55021868        LAN-GPON   
953  31/0_1_11/114/1/Multi-Service VLAN/31        7946        LAN-GPON   
954    31/0_1_9/56/1/Multi-Service VLAN/31     381 -P7        LAN-GPON   

                           Interface Information        Service Type  \
0      Frame:0/Slot:1/Port:3/ONTID:17/G

In [ ]:
print(onus)

    Operation Status Config Status  Frame  Slot  Port  ONU ID      Name  \
0           Activate        Normal      0     1     0       0      8767   
1           Activate        Normal      0     1     0       1  55021181   
2           Activate        Normal      0     1     0       2     11461   
3           Activate        Normal      0     1     0       3      8890   
4           Activate        Normal      0     1     0       4     11906   
..               ...           ...    ...   ...   ...     ...       ...   
950         Activate        Normal      0     1    15      32     11280   
951       Deactivate       Initial      0     1    15      33     11307   
952         Activate        Normal      0     1    15      34     89479   
953         Activate        Normal      0     1    15      35     11536   
954         Activate        Normal      0     1    15      37     72354   

    Vendor ID Terminal Type     Alias  ...        Line Profile  \
0        HWTC      EG8141A5     C

In [ ]:
#Funcion para separar la informacion de los puertos de esto 'Frame:0/Slot:1/Port:11/ONTID:64/GEM' en columnas separadas
def extract_values(row):
    info_parts = row.split('/')
    frame_part = info_parts[0].split(':')[1]
    slot_part = info_parts[1].split(':')[1]
    port_part = info_parts[2].split(':')[1]
    ontid_part = info_parts[3].split(':')[1]
    return frame_part, slot_part, port_part, ontid_part

# Aplicar la función a la columna 'Info' Service_port crear columnas separadas en el DataFrame
Service_port[['Frame', 'Slot', 'Port', 'ONTID']] = Service_port['Interface Information'].apply(extract_values).apply(pd.Series)
print(Service_port)

                                      Name       Alias Connection Type  \
0      33/0_1_3/17/1/Multi-Service VLAN/33       73303        LAN-GPON   
1      33/0_1_0/45/1/Multi-Service VLAN/33        8175        LAN-GPON   
2      33/0_1_0/41/1/Multi-Service VLAN/33       10561        LAN-GPON   
3      33/0_1_1/74/1/Multi-Service VLAN/33        8192        LAN-GPON   
4      33/0_1_1/14/1/Multi-Service VLAN/33        8181        LAN-GPON   
..                                     ...         ...             ...   
950   31/0_1_11/64/1/Multi-Service VLAN/31    4497 -P4        LAN-GPON   
951    31/0_1_9/28/1/Multi-Service VLAN/31  600008 -P7        LAN-GPON   
952    31/0_1_4/27/1/Multi-Service VLAN/31    55021868        LAN-GPON   
953  31/0_1_11/114/1/Multi-Service VLAN/31        7946        LAN-GPON   
954    31/0_1_9/56/1/Multi-Service VLAN/31     381 -P7        LAN-GPON   

                           Interface Information        Service Type  \
0      Frame:0/Slot:1/Port:3/ONTID:17/G

In [ ]:
#Pasar a enteros para poder comparar
Service_port['Frame'] = Service_port['Frame'].astype(int)
Service_port['Port'] = Service_port['Port'].astype(int)
Service_port['Slot'] = Service_port['Slot'].astype(int)
Service_port['ONTID'] = Service_port['ONTID'].astype(int)

Ahora que tenemos la ubicacion de las ONTs el la OLT es necesario comparar la informacion comun y extraer el numero de contrato hacia el dataset que contiene los planes de internet

In [ ]:
# Comparar Service_port con ONUs y extraer columna Contrato
def agregar_valor(row):
    matching_row = onus[
        (onus['Frame'] == row['Frame']) &
        (onus['Slot'] == row['Slot']) &
        (onus['Port'] == row['Port']) &
        (onus['ONU ID'] == row['ONTID'])
    ]
    if not matching_row.empty:
        return matching_row.iloc[0]['Name']  # Cambiar por la columna que desees
    return None

Service_port['Contrato'] = Service_port.apply(agregar_valor, axis=1)

print(Service_port)

                                      Name       Alias Connection Type  \
0      33/0_1_3/17/1/Multi-Service VLAN/33       73303        LAN-GPON   
1      33/0_1_0/45/1/Multi-Service VLAN/33        8175        LAN-GPON   
2      33/0_1_0/41/1/Multi-Service VLAN/33       10561        LAN-GPON   
3      33/0_1_1/74/1/Multi-Service VLAN/33        8192        LAN-GPON   
4      33/0_1_1/14/1/Multi-Service VLAN/33        8181        LAN-GPON   
..                                     ...         ...             ...   
950   31/0_1_11/64/1/Multi-Service VLAN/31    4497 -P4        LAN-GPON   
951    31/0_1_9/28/1/Multi-Service VLAN/31  600008 -P7        LAN-GPON   
952    31/0_1_4/27/1/Multi-Service VLAN/31    55021868        LAN-GPON   
953  31/0_1_11/114/1/Multi-Service VLAN/31        7946        LAN-GPON   
954    31/0_1_9/56/1/Multi-Service VLAN/31     381 -P7        LAN-GPON   

                           Interface Information        Service Type  \
0      Frame:0/Slot:1/Port:3/ONTID:17/G

In [ ]:
# Comparar Service_port y extraer columna serial
def agregar_valor(row):
    matching_row = onus[
        (onus['Frame'] == row['Frame']) &
        (onus['Slot'] == row['Slot']) &
        (onus['Port'] == row['Port']) &
        (onus['ONU ID'] == row['ONTID'])
    ]
    if not matching_row.empty:
        return matching_row.iloc[0]['SN']  # Cambiar por la columna que desees
    return None

Service_port['Serial'] = Service_port.apply(agregar_valor, axis=1)

print(Service_port)

                                      Name       Alias Connection Type  \
0      33/0_1_3/17/1/Multi-Service VLAN/33       73303        LAN-GPON   
1      33/0_1_0/45/1/Multi-Service VLAN/33        8175        LAN-GPON   
2      33/0_1_0/41/1/Multi-Service VLAN/33       10561        LAN-GPON   
3      33/0_1_1/74/1/Multi-Service VLAN/33        8192        LAN-GPON   
4      33/0_1_1/14/1/Multi-Service VLAN/33        8181        LAN-GPON   
..                                     ...         ...             ...   
950   31/0_1_11/64/1/Multi-Service VLAN/31    4497 -P4        LAN-GPON   
951    31/0_1_9/28/1/Multi-Service VLAN/31  600008 -P7        LAN-GPON   
952    31/0_1_4/27/1/Multi-Service VLAN/31    55021868        LAN-GPON   
953  31/0_1_11/114/1/Multi-Service VLAN/31        7946        LAN-GPON   
954    31/0_1_9/56/1/Multi-Service VLAN/31     381 -P7        LAN-GPON   

                           Interface Information        Service Type  \
0      Frame:0/Slot:1/Port:3/ONTID:17/G

In [ ]:
#Busca valores repetidos por si a algun usuario se le ha asignado mas de un plan de velocidad
valores_repetidos = Service_port[Service_port.duplicated('Contrato')]
print("Valores repetidos:")
print(valores_repetidos)

Valores repetidos:
Empty DataFrame
Columns: [Name, Alias, Connection Type, Interface Information, Service Type, Service Para, Upstream Traffic Profile, Downstream Traffic Profile, VLAN ID, Inner VLAN ID, Max. Learnable MAC Addresses, Frame, Slot, Port, ONTID, Contrato, Serial]
Index: []


In [ ]:
#Guarda la nueva informacion en google drive
Service_port.to_excel('Planes_GPON_OLT_Bello.xlsx')

In [ ]:
file_path = '/content/Planes_GPON_OLT_Bello.xlsx'
excel_dest_path = '/content/drive/MyDrive/Colab Notebooks/Planes_GPON_OLT_Bello.xlsx'
!mv "$file_path" "$excel_dest_path"